In [205]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
from tqdm import tqdm
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold 
from sklearn.decomposition import KernelPCA

In [206]:
df_train= pd.read_csv('./train_df.csv', encoding = 'euc-kr')
df_test = pd.read_csv('./test_df.csv', encoding = 'euc-kr')
df_sample = pd.read_csv('./sample_submission.csv', encoding = 'euc-kr')
place_list = pd.read_csv('./place_list')
population = pd.read_csv('./population.csv', encoding = 'euc-kr')

In [207]:
df_train

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3
...,...,...,...,...,...,...
31995,31995,5011001060063300,2635000026053400,식품,농산물,6
31996,31996,5011001095042400,1168000017002200,식품,기타식품,5
31997,31997,5011001108036200,4119700008012100,식품,농산물,9
31998,31998,5011001115011400,1132000015085100,식품,농산물,3


In [191]:
#df_train['SEND_SPG_INNB_NEW']=df_train['SEND_SPG_INNB'].apply(lambda x : str(x)[0:5])
#df_train['REC_SPG_INNB_NEW']=df_train['REC_SPG_INNB'].apply(lambda x : str(x)[0:5])
#df_test['SEND_SPG_INNB_NEW']=df_test['SEND_SPG_INNB'].apply(lambda x : str(x)[0:5])
#df_test['REC_SPG_INNB_NEW']=df_test['REC_SPG_INNB'].apply(lambda x : str(x)[0:5])

#place_list['번호']=place_list['hi'].str.split('_').str[0]
#place_list['지역']=place_list['hi'].str.split('_').str[1]

df_train.rename({'DL_GD_LCLS_NM':'제품_대분류', 'DL_GD_MCLS_NM':'제품_소분류','INVC_CONT':'배송량','SEND_SPG_INNB_NEW':'배송지',
                 'REC_SPG_INNB_NEW':'수하지'}, axis = 1, inplace=True)
df_test.rename({'DL_GD_LCLS_NM':'제품_대분류', 'DL_GD_MCLS_NM':'제품_소분류','SEND_SPG_INNB_NEW':'배송지',
                'REC_SPG_INNB_NEW':'수하지'}, axis = 1, inplace=True)
df_train
def func_drop(df, df1):
    population.drop(['2020. 01.1','2020. 01.2'], axis = 1, inplace= True)
    population.drop([0,1], inplace= True)
    place_list.drop('hi', axis = 1,inplace =True)
    df_train.drop(['SEND_SPG_INNB','REC_SPG_INNB','index'], axis = 1, inplace=True)
    df_test.drop(['SEND_SPG_INNB','REC_SPG_INNB','index'], axis = 1, inplace= True)

def df2dict(df):
    df_dict = {}
    for k, v in zip(df.iloc[:, 0], df.iloc[:, 1]):
        df_dict[k] = v
    return df_dict

def df2dict3(df):
    df_dict = {}
    for k, v in zip(df.loc[:, '이동구역'], df.loc[:, '평균_배송량']):
        df_dict[k] = v
    return df_dict

def exchange_value(df, df1):
    result = [0 for i in range(32000)]
    for a, i in enumerate(df):
        for j,v in df1.items():
            if i ==j:
                result[a]=v
    return result

def exchange_value2(df, df1):
    result = [0 for i in range(4640)]
    for a, i in enumerate(df):
        for j,v in df1.items():
            if i ==j:
                result[a]=v
    return result

func_drop(population, place_list)
place_dict = df2dict(place_list)

df_train['배송지'] = exchange_value(df_train['배송지'], place_dict)
df_train['수하지'] = exchange_value(df_train['수하지'], place_dict)
df_test['배송지'] = exchange_value2(df_test['배송지'], place_dict)
df_test['수하지'] = exchange_value2(df_test['수하지'], place_dict)

In [192]:
def data_split():
    df_train['배송지']=df_train['배송지'].str.replace(' ','')
    df_train['수하지']=df_train['수하지'].str.replace(' ','')
    df_test['배송지']=df_test['배송지'].str.replace(' ','')
    df_test['수하지']=df_test['수하지'].str.replace(' ','')
    df_train['배송지_NEW'] = df_train['배송지'].apply(lambda x : x[3:6] if len(x)==6 else x[3:7] if len(x)==7 else x)
    df_train['수하지_NEW'] = df_train['수하지'].apply(lambda x : x[3:6] if len(x)==6 else x[3:7] if len(x)==7 else x)
    df_test['배송지_NEW'] = df_test['배송지'].apply(lambda x : x[3:6] if len(x)==6 else x[3:7] if len(x)==7 else x)
    df_test['수하지_NEW'] = df_test['수하지'].apply(lambda x : x[3:6] if len(x)==6 else x[3:7] if len(x)==7 else x)
    df_train.drop(['배송지','수하지'], axis = 1, inplace=True)
    df_test.drop(['배송지','수하지'], axis =1, inplace =True)
        
data_split()

In [193]:
def add_population(df,df1, df2):
    df_copy = df2dict(df2)
    df['배송지_인구']= exchange_value(df['배송지_NEW'], df_copy)
    df['수하지_인구']= exchange_value(df['수하지_NEW'], df_copy)
    df1['배송지_인구']= exchange_value2(df1['배송지_NEW'], df_copy)
    df1['수하지_인구']= exchange_value2(df1['수하지_NEW'], df_copy)
    
    return df, df1
df_train, df_test = add_population(df_train,df_test, population)

In [194]:
def add_population(df):
    a =['수하지_인구','배송지_인구']
    column_list =['수하지_NEW','배송지_NEW']
    for u, j in zip(a,column_list):
        for i,v in enumerate(df[j]):
            if v =='포항시북구':
                df.loc[i,u]=272494
            if v =='포항시남구':
                df.loc[i,u]=253700
            if v =='청원군':
                df.loc[i,u]=159486
            if v =='창원시마산회원구':
                df.loc[i,u]=187000
            if v =='창원시마산합포구':
                df.loc[i,u]=181226
            if v =='원미구':
                df.loc[i,u]=442000
            if v =='오정구':
                df.loc[i,u]=187779
            if v =='소사구':
                df.loc[i,u]=221647
            if v =='별자치시':
                df.loc[i,u]=275600

In [195]:
add_population(df_train)
add_population(df_test)
df_test.loc[df_test['수하지_NEW']=='별자치시',:]

,제품_대분류,제품_소분류,배송지_NEW,수하지_NEW,배송지_인구,수하지_인구
7,식품,농산물,서귀포시,별자치시,181547,275600
267,식품,수산,제주시,별자치시,489202,275600
416,식품,농산물,서귀포시,별자치시,181547,275600
604,식품,농산물,제주시,별자치시,489202,275600
643,식품,농산물,서귀포시,별자치시,181547,275600
878,식품,수산,제주시,별자치시,489202,275600
1125,식품,수산,제주시,별자치시,489202,275600
1768,여행/문화,문화컨텐츠,제주시,별자치시,489202,275600
2473,식품,농산물,서귀포시,별자치시,181547,275600
2575,식품,농산물,제주시,별자치시,489202,275600


In [196]:
# df_train['이동구역']= df_train[['배송지_NEW','수하지_NEW']].apply(lambda x : str(x[0])+'_'+x[1], axis = 1)
# df_train['평균_배송량'] = df_train.groupby('이동구역').배송량.transform('mean')
# df_test['이동구역'] = df_test[['배송지_NEW','수하지_NEW']].apply(lambda x : str(x[0])+'_'+x[1], axis = 1)
# exchange_value3 = df2dict3(df_train)
# df_test['평균_배송량']= exchange_value2(df_test['이동구역'], exchange_value3)
# scaler = StandardScaler()
# df_train[['배송지_인구','수하지_인구','평균_배송량']]=scaler.fit_transform(df_train[['배송지_인구','수하지_인구','평균_배송량']])

In [197]:
class MultiLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
df_train = MultiLabelEncoder(columns = ['배송지_NEW','수하지_NEW','제품_대분류','제품_소분류']).fit_transform(df_train)
df_test = MultiLabelEncoder(columns = ['배송지_NEW','수하지_NEW','제품_대분류','제품_소분류']).transform(df_test)

In [198]:
df_train['배송지_인구']=pd.to_numeric(df_train['배송지_인구'])
df_train['수하지_인구']=pd.to_numeric(df_train['수하지_인구'])
X_features = df_train.drop('배송량', axis=1)
y_target = df_train['배송량']
X_features.shape, y_target.shape

((32000, 6), (32000,))

In [199]:
df_train

,제품_대분류,제품_소분류,배송량,배송지_NEW,수하지_NEW,배송지_인구,수하지_인구
0,4,12,3,101,179,442471,489202
1,1,11,3,49,179,532662,489202
2,4,6,9,49,179,532662,489202
3,2,8,10,36,179,233066,489202
4,2,0,3,99,179,431050,489202
...,...,...,...,...,...,...,...
31995,2,8,6,169,220,489202,406312
31996,2,5,5,169,1,489202,545938
31997,2,8,9,169,108,489202,221647
31998,2,8,3,169,62,489202,332753


In [200]:
df_test

,제품_대분류,제품_소분류,배송지_NEW,수하지_NEW,배송지_인구,수하지_인구
0,2,8,65,103,181547,431050
1,2,8,65,38,181547,233066
2,2,8,65,117,181547,475396
3,2,8,65,109,181547,81840
4,2,8,65,111,181547,428770
...,...,...,...,...,...,...
4635,2,8,65,96,181547,99814
4636,2,8,65,97,181547,481174
4637,2,8,65,94,181547,179626
4638,2,8,65,210,181547,273587


In [201]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 511)

for trn_idx, val_idx in folds.split(X_features) :
    X_train, y_train = X_features.iloc[trn_idx], y_target.iloc[trn_idx] 
    X_val, y_val = X_features.iloc[val_idx], y_target.iloc[val_idx]
    
    params = {'learning_rate': 0.01, 
              'max_depth': 16, 
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'rmse', 
              'is_training_metric': True, 
              'num_leaves': 144, 
              'feature_fraction': 0.9, 
              'bagging_fraction': 0.7, 
              'bagging_freq': 5, 
              'seed':2019}

    train_ds = lgb.Dataset(X_train, label=y_train) 
    valid_ds = lgb.Dataset(X_val, label=y_val) 
    
    model = lgb.train(params, train_ds, 3000, valid_ds, verbose_eval=100, early_stopping_rounds=100) 

C:\Users\user\anaconda3\envs\caba\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\user\anaconda3\envs\caba\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000460 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 843
[LightGBM] [Info] Number of data points in the train set: 25600, number of used features: 6
[LightGBM] [Info] Start training from score 4.730234
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 6.05594
[200]	valid_0's rmse: 5.99345
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[100]	valid_0's rmse: 4.90925
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	valid_0's rmse: 4.87146
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[400]	valid_0's rmse: 6.49514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1200]	valid_0's rmse: 6.24181
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1600]	valid_0's rmse: 6.207
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2100]	valid_0's rmse: 6.1701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [202]:
df_test['배송지_인구']=pd.to_numeric(df_test['배송지_인구'])
df_test['수하지_인구']=pd.to_numeric(df_test['수하지_인구'])

In [203]:
df_test

,제품_대분류,제품_소분류,배송지_NEW,수하지_NEW,배송지_인구,수하지_인구
0,2,8,65,103,181547,431050
1,2,8,65,38,181547,233066
2,2,8,65,117,181547,475396
3,2,8,65,109,181547,81840
4,2,8,65,111,181547,428770
...,...,...,...,...,...,...
4635,2,8,65,96,181547,99814
4636,2,8,65,97,181547,481174
4637,2,8,65,94,181547,179626
4638,2,8,65,210,181547,273587


In [204]:
pred = model.predict(df_test,predict_disable_shape_check=True)
df_sample['INVC_CONT'] = pred
df_sample.to_csv('baseline2.csv',index = False)

In [124]:
print(list(X_train.columns))

['REC_SPG_INNB_NEW', 'SEND_SPG_INNB_NEW', 'DL_GD_LCLS_NM_디지털/가전', 'DL_GD_LCLS_NM_생활/건강', 'DL_GD_LCLS_NM_식품', 'DL_GD_LCLS_NM_여행/문화', 'DL_GD_LCLS_NM_패션의류', 'DL_GD_LCLS_NM_화장품/미용', 'DL_GD_MCLS_NM_가공식품', 'DL_GD_MCLS_NM_건강식품', 'DL_GD_MCLS_NM_건강용품', 'DL_GD_MCLS_NM_과자', 'DL_GD_MCLS_NM_기타디지털/가전', 'DL_GD_MCLS_NM_기타식품', 'DL_GD_MCLS_NM_기타패션의류', 'DL_GD_MCLS_NM_냉동/간편조리식품', 'DL_GD_MCLS_NM_농산물', 'DL_GD_MCLS_NM_문구/사무용품', 'DL_GD_MCLS_NM_문화컨텐츠', 'DL_GD_MCLS_NM_반려동물', 'DL_GD_MCLS_NM_상의', 'DL_GD_MCLS_NM_생활용품', 'DL_GD_MCLS_NM_수산', 'DL_GD_MCLS_NM_스킨케어', 'DL_GD_MCLS_NM_음료', 'DL_GD_MCLS_NM_음반', 'DL_GD_MCLS_NM_주방용품', 'DL_GD_MCLS_NM_축산']


In [121]:
len(submission2.columns)

25

In [127]:
len(X_train.columns)

28

In [3]:
df_train= pd.read_csv('./TC_NU_SPG_50_METER/TC_NU_SPG_50_METER_11.csv')

In [36]:
df_train

,격자공간고유번호,격자공간명,시군구코드,시군구명
0,1150000001001100,다사3855,11500,강서구
1,1150000001001200,다사3855,11500,강서구
2,1150000001001300,다사3855,11500,강서구
3,1150000001001400,다사3855,11500,강서구
4,1150000001002100,다사3855,11500,강서구
...,...,...,...,...
241995,1174000025099400,다사6846,11740,강동구
241996,1174000025100100,다사6846,11740,강동구
241997,1174000025100200,다사6846,11740,강동구
241998,1174000025100300,다사6846,11740,강동구


In [4]:
import os
path = './TC_NU_SPG_50_METER/'
file_list = os.listdir(path)
print(file_list)

u = []
for file in tqdm(file_list):
    data = pd.read_csv(path+file)
    u.append(data[['시군구코드','시군구명']].apply(lambda x : str(x[0])+'_'+x[1], axis = 1))

['TC_NU_SPG_50_METER_11.csv', 'TC_NU_SPG_50_METER_26.csv', 'TC_NU_SPG_50_METER_27.csv', 'TC_NU_SPG_50_METER_28.csv', 'TC_NU_SPG_50_METER_29.csv', 'TC_NU_SPG_50_METER_30.csv', 'TC_NU_SPG_50_METER_31.csv', 'TC_NU_SPG_50_METER_36.csv', 'TC_NU_SPG_50_METER_41.csv', 'TC_NU_SPG_50_METER_42.csv', 'TC_NU_SPG_50_METER_43.csv', 'TC_NU_SPG_50_METER_44.csv', 'TC_NU_SPG_50_METER_45.csv', 'TC_NU_SPG_50_METER_46.csv', 'TC_NU_SPG_50_METER_47.csv', 'TC_NU_SPG_50_METER_48.csv', 'TC_NU_SPG_50_METER_50.csv']


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [07:41<00:00, 27.13s/it]


In [12]:
result = []
def fuck(df):
    for i in df:
        result.append(list(set(i)))
fuck(u)

In [69]:
result
answer = sum(result, [])
palce_list = pd.DataFrame(data = answer, columns = ['hi'])
palce_list.to_csv('./place_list', header = True, index = None)

In [148]:
df_train2= pd.read_csv('./train_df.csv', encoding = 'euc-kr')
df_test2 = pd.read_csv('./test_df.csv', encoding = 'euc-kr')
df_sample = pd.read_csv('./sample_submission.csv', encoding = 'euc-kr')
place_list = pd.read_csv('./place_list')
population = pd.read_csv('./population.csv', encoding = 'euc-kr')

In [152]:
df_train2
encoder = ce.target_encoder.TargetEncoder(cols =['DL_GD_LCLS_NM'])
encoder.fit(df_train2['DL_GD_LCLS_NM'],df_train2['INVC_CONT'])
df_train2['new'] = encoder.transform(df_train2['DL_GD_LCLS_NM'])
df_train2

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,new
0,0,1129000014045300,5011000220046300,패션의류,상의,3,6.678694
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3,5.407921
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9,6.678694
3,3,1154500002014200,5011000315087400,식품,농산물,10,4.658195
4,4,1165000021008300,5011000177051200,식품,가공식품,3,4.658195
...,...,...,...,...,...,...,...
31995,31995,5011001060063300,2635000026053400,식품,농산물,6,4.658195
31996,31996,5011001095042400,1168000017002200,식품,기타식품,5,4.658195
31997,31997,5011001108036200,4119700008012100,식품,농산물,9,4.658195
31998,31998,5011001115011400,1132000015085100,식품,농산물,3,4.658195


In [153]:
encoder = ce.target_encoder.TargetEncoder(cols =['DL_GD_MCLS_NM'])
encoder.fit(df_train2['DL_GD_MCLS_NM'],df_train2['INVC_CONT'])
df_train2['new2'] = encoder.transform(df_train2['DL_GD_MCLS_NM'])

In [154]:
df_train2

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,new,new2
0,0,1129000014045300,5011000220046300,패션의류,상의,3,6.678694,3.672897
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3,5.407921,4.444134
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9,6.678694,10.375479
3,3,1154500002014200,5011000315087400,식품,농산물,10,4.658195,4.297401
4,4,1165000021008300,5011000177051200,식품,가공식품,3,4.658195,6.807151
...,...,...,...,...,...,...,...,...
31995,31995,5011001060063300,2635000026053400,식품,농산물,6,4.658195,4.297401
31996,31996,5011001095042400,1168000017002200,식품,기타식품,5,4.658195,6.130548
31997,31997,5011001108036200,4119700008012100,식품,농산물,9,4.658195,4.297401
31998,31998,5011001115011400,1132000015085100,식품,농산물,3,4.658195,4.297401


In [51]:
kpca = KernelPCA(kernel = 'rbf')
kpca.fit_transform(df_train)

KeyboardInterrupt: 

In [ ]:
cumsum = np.cumsum(kpca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95)+1
print('선택할 차원 수: ',d)

In [152]:
'''from collections import defaultdict
class MultiColLabelEncoder:
    def __init__(self):
        self.encoder_dict = defaultdict(LabelEncoder)
    def fit_transform(self, X:pd.DataFrame, columns :list):
        if not isinstance(columns, list):
            columns = [columns]
        output = X.copy()
        output[columns] = X[columns].apply(lambda x :self.encoder_dict[x.name].fit_transform(x))
        
        return output
        '''

In [153]:
mcle = MultiColLabelEncoder()
transformed_data = mcle.fit_transform(df_train, columns=['배송지_NEW','수하지_NEW'])

In [165]:
df_train = MultiLabelEncoder(columns = ['배송지_NEW','수하지_NEW']).fit_transform(df_train)
df_test = MultiLabelEncoder(columns = ['배송지_NEW','수하지_NEW']).transform(df_test)

In [167]:
df_train = MultiLabelEncoder(columns = ['제품_대분류','제품_소분류']).fit_transform(df_train)
df_test = MultiLabelEncoder(columns = ['제품_대분류','제품_소분류']).transform(df_test)

In [168]:
df_train

,제품_대분류,제품_소분류,배송량,배송지_NEW,수하지_NEW,배송지_인구,수하지_인구
0,4,12,3,101,179,442471,489202
1,1,11,3,49,179,532662,489202
2,4,6,9,49,179,532662,489202
3,2,8,10,36,179,233066,489202
4,2,0,3,99,179,431050,489202
...,...,...,...,...,...,...,...
31995,2,8,6,169,220,489202,406312
31996,2,5,5,169,1,489202,545938
31997,2,8,9,169,108,489202,221647
31998,2,8,3,169,62,489202,332753
